# Input Widgets and Geospatial Data Analysis

In addition to ipywidgets 5.0.0, and pythreejs, this example requires GDAL

**GDAL installation:**

```bash
conda install gdal
conda install krb5
```

In [1]:
from __future__ import print_function
from ipywidgets import Controller, FloatText, HTML, VBox
from pythreejs.install import install
from traitlets.traitlets import link, dlink

# Terrain modeling? Let us check out the Grand Canyon

In [2]:
from pythreejs import *
import numpy as np
import gdal as gd

In [3]:
gc_ds = gd.Open('gc_dem.tif')
dem = gc_ds.ReadAsArray()[::20, ::20]
gt = gc_ds.GetGeoTransform()

z = (dem - np.mean(dem)) / 1000
nx, ny = z.shape

surf_g = SurfaceGeometry(z=list(z.flat), height_segments=nx - 1, width_segments=ny - 1)
surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z[::-1], colormap='terrain')), scale=(10, 10, 1))
scene = Scene(children=[AmbientLight(color='#777777'),
                        surf, 
                        DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

In [4]:
c = PerspectiveCamera(position=[0, 10, 10], up=[0, 0, 1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)],
                      aspect=2)
width = 950
height = 950 / c.aspect
c.look_at(c.position, (1, 0, 0))
fly_controls = FlyControls(controlling=c)
renderer = Renderer(camera=c, scene=scene, width=str(width), height=str(height), controls=[fly_controls])

In [5]:
c.position = [0, 10, 10]
c.look_at(c.position, (1, 0, 0))

In [6]:
renderer

A Jupyter Widget

## Adjusting the camera position?

In [7]:
c.position = [10, 15, 10]
c.look_at(c.position, (1, 0, 0))

<img src="Flow.svg"></img>

# Sliders?

In [8]:
from ipywidgets import FloatSlider, HBox, VBox

x_slider, y_slider, z_slider = (FloatSlider(description='x', min=10.0, max=20.0, orientation='vertical'),
                                FloatSlider(description='y', min=10.0, max=20.0, orientation='vertical'),
                                FloatSlider(description='z', min=10.0, max=20.0, orientation='vertical'))

In [9]:
def update(change):
    c.position = [x_slider.value, y_slider.value, z_slider.value]
    
x_slider.observe(update, names=['value'])
y_slider.observe(update, names=['value'])
z_slider.observe(update, names=['value'])

In [10]:
HBox([x_slider, y_slider, z_slider, renderer])

A Jupyter Widget

In [11]:
pad = Controller()
pad

A Jupyter Widget

In [15]:
pad.axes[1].value

0.0

In [16]:
factor = 10
def affine(constant, factor):
    return lambda x: constant + factor * x

pad.links = []

def setup():
    if pad.connected:
        pad.links.append(dlink((pad.axes[1], 'value'), (fly_controls, 'pitch'), affine(0.0, factor)))
        pad.links.append(dlink((pad.axes[0], 'value'), (fly_controls, 'roll'), affine(0.0, -factor)))
        pad.links.append(dlink((pad.axes[3], 'value'), (fly_controls, 'forward_speed'), affine(0.0, 2 * factor)))
        pad.links.append(dlink((pad.axes[2], 'value'), (fly_controls, 'yaw'), affine(0.0, factor)))
    if not pad.connected:
        for l in pad.links:
            l.unlink()
        pad.links = []

pad.observe(setup, names=['connected'])
setup()